# ファイル操作

In [1]:
import os
import shutil
import subprocess

* カレントディレクトリをホームに変更

In [2]:
os.chdir(os.path.expanduser("~"))

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [3]:
os.mkdir("Empty")

* 一時ディレクトリの場合
```Python
import tempfile
t=tempfile.TemporaryDirectory()
t.name # ディレクトリのパス
t.cleanup() # ディレクトリを削除する
```

* Blankという空ファイルを作成

In [4]:
# 既にBlankが存在していたら内容が削除される
# 削除したくない場合は, "w" を "r" や "a" にする
io=open("Blank","w")
io.close()

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [5]:
os.symlink("../Blank","Empty/Symlink")

- ハードリンクの場合
```Python
os.link("../Blank","Empty/Link")
```

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [6]:
# 既にUntitled.mdが存在すれば上書きされる
io=open("Untitled.md","w")
io.write("# Header 1")
io.close()

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [7]:
shutil.move("Empty","Package")
# os.rename("Empty","Package")

'Package'

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [8]:
shutil.move("Package/Symlink","Package/Alias")
# os.rename("Package/Symlink","Package/Alias")

'Package/Alias'

* Untitled.mdを移動して,名称変更

In [9]:
shutil.move("Untitled.md","Package/Headers.md")
# os.rename("Untitled.md","Package/Headers.md")

'Package/Headers.md'

- `os.rename` でファイル移動するとエラーになる場合があるみたい

## 追記
* Markdownファイルに追記

In [10]:
io=open("Package/Headers.md","a")
io.write("\n## Header 2\n### Header 3")
io.close()

## 読込み
* Markdownファイルを読込み

In [11]:
io=open("Package/Headers.md","r")
print(io.read())
io.close()

# Header 1
## Header 2
### Header 3


## 再帰的にフォルダ作成
* フォルダを一気に作成

In [12]:
os.makedirs("Package/Module/Submodule/Item")

- `os.mkdir` の代わりに `os.makedirs` を使うことで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [13]:
os.listdir("Package")

['Alias', 'Module', 'Headers.md']

- ワイルドカードを使って,条件を満たすファイルを見つけ出す場合は

```Python
import glob
glob.glob("*.md")
glob.glob("*.md", recursive=True)
```

- ファイルパスを指定して条件に合致するものを絞り込む  
	`recursive=True` とすればディレクトリの内部まで検索する

## ファイル/フォルダの複製
* BlankとModuleを複製

In [14]:
shutil.copy("Blank","Package/Blank")
shutil.copytree("Package/Module","Package/Module Copy")

'Package/Module Copy'

- `shutil.copytree` を用いることで,フォルダ内の内容もまとめて複製できる

## ファイル/フォルダの削除

* ファイルBlankを削除

In [15]:
os.remove("Blank")
# os.unlink("Blank")

* シンボリックリンクAliasを削除

In [16]:
os.remove("Package/Alias")
# os.unlink("Package/Alias")

* フォルダModuleを削除

In [17]:
shutil.rmtree("Package/Module")

- 空のフォルダは `os.rmdir` で削除できる。空でない場合は削除できない。  
	`shutil.rmtree` を使えば,空でなくても削除できる

## 権限を確認/変更

* 状態を確認する関数

In [18]:
def check(path):
	if os.path.exists(path): # 或いは os.access(pt,os.F_OK)
		print("存在しています")
	else:
		print("存在していません")

	if os.path.islink(path):
		print("シンボリックリンクです")
		print(f"リンク先: {os.readlink(path)}")
		if os.path.isfile(path):
			print("リンク先はファイルです")
		if os.path.isdir(path):
			print("リンク先はフォルダです")
	else:
		if os.path.isfile(path):
			print("ファイルです")
		if os.path.isdir(path):
			print("フォルダです")
	if os.access(path,os.R_OK):
		print("読込可能です")
	if os.access(path,os.W_OK):
		print("書込可能です")
	if os.access(path,os.X_OK):
		print("実行可能です")

	if os.path.getsize(path)==0:
		print("空です")

* Markdownファイルの現在の状態を確認

In [19]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
書込可能です


* 読込/書込権限を剥奪

In [20]:
os.chmod("Package/Headers.md",0o044)

* 状態を確認

In [21]:
check("Package/Headers.md")

存在しています
ファイルです


* 読込/実行権限を付加

In [22]:
os.chmod("Package/Headers.md",0o544)

* 状態を確認

In [23]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
実行可能です


- `os.chmod` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	Pythonでは8進数で表記するために,前に `0o` を付加する

	e.g. `os.chmod(somefile,0o754)`
	* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
	* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
	* その他は 4 だから,読込のみ可能